In [1]:
%load_ext autoreload 
%autoreload 2
import sys
sys.path.append('/home/anhphantq/otto')

In [2]:
#lb
import pandas as pd
edges = pd.concat([
    pd.read_parquet(f'/home/anhphantq/unidata_sub/edges_sub_frag_{i}_v1.parquet') for i in range (10) 
    ])
edges_test = pd.read_parquet('/home/anhphantq/unidata_sub/edges_test_sub_v1.parquet')

vertices = pd.read_parquet('/home/anhphantq/unidata_sub/train_vprops_v1.parquet')
vertices_test = pd.read_parquet('/home/anhphantq/unidata_sub/test_vprops_v1.parquet')

In [2]:
#valid
import pandas as pd
edges = pd.concat([
    pd.read_parquet(f'/home/anhphantq/splitted_unidata_sub/edges_sub_frag_{i}_v1.parquet') for i in range (10) 
    ])
edges_test = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/edges_test_sub_v1.parquet')

vertices = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/train_vprops_v1.parquet')
vertices_test = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/test_vprops_v1.parquet')

In [3]:
from xgb_utils import feature_id_map
import numpy as np 
disk_vp = np.memmap('splitted_vertiec_properties.array', dtype='float64', mode='r+').reshape(len(feature_id_map), -1)


In [3]:
from xgb_utils import feature_id_map
import numpy as np 
disk_vp = np.memmap('vertiec_properties.array', dtype='float64', mode='r+').reshape(len(feature_id_map), -1)


In [4]:
JOIN = True

In [4]:
JOIN = False

In [5]:
#Params
import numpy as np
TEST_CLICK_WEIGHT = 3
TEST_CART_WEIGHT = 6
TEST_ORDER_WEIGHT_L= 6
TEST_ORDER_WEIGHT_S = 2

MAX_NUM_CANDS = 100

MIN_TOPK_COUSER = 1000
COFITNESS_THRES = 0


ALPHA = 0.5
BETA = 0.5
FILTER = False

#PARAMS

get_test_items_params = {
      'test_click_weight': TEST_CLICK_WEIGHT,
      'test_cart_weight': TEST_CART_WEIGHT,
      'test_order_weight_l': TEST_ORDER_WEIGHT_L,
      'test_order_weight_s': TEST_ORDER_WEIGHT_S,
      'day' : 3600 * 8,  
  }

get_items_params = {
      'click_weight': np.array([12,3,1]).reshape(-1,1),
      'cart_weight' : np.array([12, 6, 10]).reshape(-1,1), 
      'order_weight': np.array([6,1,1]).reshape(-1,1), 
      'day' : 3600 * 8,  
    }



# Create edges and vertices

In [6]:
first_test_session = edges_test.iloc[0,0]

In [7]:
all_edges = pd.concat((edges, edges_test))

In [8]:
del edges, edges_test

In [9]:
all_vertices = pd.concat((vertices, vertices_test))

In [10]:
del vertices, vertices_test

In [11]:
all_edges

,"('session', '')","('aid', '')","('ts', 'last')","('ts', 'first')","('clicks', 'sum')","('carts', 'sum')","('orders', 'sum')","('subsession', 'encode')"
0,0,6851,2022018,2022018,1,0,0,35184372088832
1,0,7563,2031353,2031315,2,0,0,70368744177664
2,0,10268,1853151,1853151,1,0,0,2199023255552
3,0,16246,62919,62919,1,0,0,2
4,0,30373,1798887,1798887,1,0,0,1099511627776
...,...,...,...,...,...,...,...,...
5006552,14571577,1141710,3023974,3023974,1,0,0,1
5006553,14571578,519105,3023975,3023975,1,0,0,1
5006554,14571579,739876,3023975,3023975,1,0,0,1
5006555,14571580,202353,3023981,3023981,1,0,0,1


In [12]:
all_vertices

,"('session', '')","('ts', 'min')","('ts', 'max')","('ts', 'count')","('subsession', 'max')","('consistency', '')"
0,0,1659304800,1661684983,237,60,2380183
1,1,1659304800,1661714854,32,10,2410054
2,2,1659304800,1661714215,33,4,2409415
3,3,1659304800,1661109666,177,25,1804866
4,4,1659304800,1661586681,19,3,2281881
...,...,...,...,...,...,...
1671798,14571577,1662328774,1662328774,1,0,0
1671799,14571578,1662328775,1662328775,1,0,0
1671800,14571579,1662328775,1662328775,1,0,0
1671801,14571580,1662328781,1662328781,1,0,0


In [13]:
all_edges.iloc[:,1] = all_edges.iloc[:,1] + 100000000


In [14]:
all_vertices = all_vertices.values

In [15]:
import numpy as np
all_edges = all_edges.values

# Load to graph

In [16]:
import graph_tool

In [17]:
from graph_tool import Graph
ug = Graph(directed=False)

In [18]:
vmap = ug.add_edge_list(all_edges[:, : 2].astype(np.uint32))

In [19]:
#short type
all_edges[:, 4 : 7] = np.clip (all_edges[:, 4 : 7], 0,  32_767)

In [20]:
data_type = {
    'elts': 'unsigned int',
    'efts': 'unsigned int',
    'eclick': 'short',
    'ecart': 'short',
    'eorder': 'short',
    'esub': 'unsigned long'
}

In [21]:
for i, prop in enumerate(['elts', 'efts', 'eclick', 'ecart', 'eorder', 'esub']):
  setattr(ug.ep, prop, ug.new_ep(data_type[prop], vals = all_edges[:, i + 2]))  


In [22]:
del all_edges

In [23]:
ug.num_edges()

128380808

In [24]:
elts = ug.ep['elts']
efts = ug.ep['efts']
eclick = ug.ep['eclick']
ecart = ug.ep['ecart']
eorder = ug.ep['eorder']
esub = ug.ep['esub']


In [24]:
vp = {}
vp['num_sub'] = ug.new_vp('double',  all_vertices[:, -2] ).get_array()
vp['consistency'] = ug.new_vp('double',  1 / (1 + np.log(all_vertices[:, -1] + 1)) ).get_array()
vp['num_actions'] = ug.new_vp('unsigned int', all_vertices[:, -3]).get_array()

In [25]:
del all_vertices

# Count actions

In [ ]:
from graph_tool import incident_edges_op
vp['num_clicks'] = incident_edges_op(ug, "out", "sum", eclick).get_array()
vp['num_carts'] = incident_edges_op(ug, "out", "sum", ecart).get_array()
vp['num_orders'] = incident_edges_op(ug, "out", "sum", eorder).get_array()

# Pagerank

In [27]:
%%time
import graph_tool.centrality as gt
vp['pr'] = gt.pagerank(ug).get_array()

CPU times: user 13min 39s, sys: 2.1 s, total: 13min 41s
Wall time: 4min 5s


# Vertice pruning

In [ ]:
from tqdm import tqdm
k = 0
vertices_filter = np.ones(ug.num_vertices(), dtype = bool)
for i, v in tqdm(enumerate(ug.vertices()), total = first_test_session):
  # if k == 100:
  #   break
  # k+=1
  if v == first_test_session:
    break
  if v.out_degree() < 2:
    vertices_filter[i] = 0

In [ ]:
np.sum(~vertices_filter)

1381909

In [ ]:
from graph_tool import GraphView
fug = GraphView(ug, vfilt = vertices_filter)

In [ ]:
fug.num_edges()

98511887

# Create Disk VP

In [28]:
for k in vp.keys():
  vp[k] = np.array(vp[k])

In [29]:
import multiprocessing
mem_size = (len(vp.keys()), vp[k].shape[0])
disk_vp = np.memmap('vertiec_properties.array', dtype='float64', mode='w+', shape=(mem_size[0],mem_size[1]))


In [30]:
for i, k in enumerate(vp.keys()):
  disk_vp[i] = vp[k] 


In [31]:
del vp

# Load test

In [26]:
ROOT = '/home/anhphantq/otto'
DATA_FOLDER = ROOT + '/data'



In [27]:
ROOT = '/home/anhphantq/otto'
DATA_FOLDER = ROOT + '/splitted_data'



In [27]:
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')

In [28]:
import numpy as np
session_list = np.sort(test_df['session'].unique())


In [46]:
session_list = session_list[:len(session_list) // 20]

In [30]:
session_list = session_list[len(session_list) // 20 : len(session_list) // 5]
# session_list = np.random.permutation(session_list)
# session_list = session_list[:len(session_list) // 20]

In [31]:
# session_list = session_list[len(session_list) // 20 : ]

In [29]:
len(session_list)

1671803

In [30]:
del test_df

# Find neighbour strategy

In [31]:
from xgb_utils import USER_FEATURES, ITEM_FEATURES, feature_id_map

def get_user_features(user):
  return [disk_vp[feature_id_map[f]][user] for f in USER_FEATURES]

def get_item_features(items):

  return [
      np.array([
          disk_vp[feature_id_map[f]][item] for item in items 
      ]) for f in ITEM_FEATURES
  ]

def get_test_items(user, beta):
  global ug, fug, elts, eclick, ecart, eorder, get_test_items_params, FILTER
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder 6 subsession, 7 ftss
  edges = (fug if FILTER else ug).get_all_edges(user, eprops = [elts, eclick, ecart, eorder, esub, efts])
        
  return _get_test_items(edges, beta, **get_test_items_params)


def get_user_neighbours(item):
  global ug, fug, elts, efts, eclick, ecart, eorder, esub, FILTER
  #1 target 2 ltss 3 ftss 4 click 5 cart 6 order 7 subsession
  edges = (fug if FILTER else ug).get_all_edges(item, eprops = [elts, efts, eclick, ecart, eorder, esub])
  return edges[:,1], edges[:,2], edges[:,3], edges[:, 4], edges[:, 5], edges[:, 6], edges[:, 7]
  

def get_item_neighbours(user, lts, fts, prevsub, prev_click, prev_cart, prev_order, alpha):
  global ug, fug, elts, efts, eclick, ecart, eorder, esub, get_items_params, FILTER
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order, 7 subsession

  edges = (fug if FILTER else ug).get_all_edges(user, eprops = [elts, efts, eclick, ecart, eorder, esub])

  return _get_item_neighbours(edges, lts, fts, prev_click = prev_click, prev_cart = prev_cart, prev_order = prev_order
                              , alpha = alpha, prevsub = prevsub, **get_items_params)
  


# Numba funcs

In [32]:
import numba as nb
import numpy as np
from numba import prange
    
@nb.njit(nb.float64[:](nb.int64[:]), cache = True, nogil = True)
def reverse_normalize(X):
  m = np.max(X)
  return (m - X) / (m - np.min(X) + 1e-12) 

@nb.njit(nb.float64[:](nb.int64[:]), cache = True, nogil = True)
def normalize(X):
  m = np.max(X)
  return (X - np.min(X)) / (m - np.min(X) + 1e-12) 

@nb.njit([nb.int64[:](nb.float64[:]),
          nb.int64[:](nb.int64[:]),
          ], cache = True, nogil = True)
def get_rank(x):
  return np.argsort(np.argsort(x))

@nb.njit(nb.int64(nb.int64), cache = True, nogil = True)
def decode_1d(x):
  rs = 0
  while x > 0:
    if x % 2 == 1:
      rs += 1
    x = x // 2
  return rs

@nb.njit(nb.int64[:](nb.int64[:]), cache = True, nogil = True)
def decode_2d(x):
  return np.array([decode_1d(i) for i in x])



@nb.njit(nb.float64[:, :](nb.float64[:,:]))
def cumsum_2d(x):
  result = np.empty_like(x, dtype = np.float64)
  result[:, 0] = x[:, 0]
  for i in prange(1, x.shape[1]):
    result[:, i] = x[:, i] +  result[:, i - 1]
  return result


@nb.njit([nb.float64[:](nb.float64[:], nb.float64, nb.float64),
          nb.float64[:](nb.float64[:], nb.int64, nb.float64),
          nb.float64[:](nb.float64[:], nb.float64, nb.int64),
         nb.float64[:](nb.float64[:], nb.int64, nb.int64)], cache = True, nogil = True)
def normalize_to_range(X, l, r):
  m = np.min(X)
  return (X - m) / (np.max(X) - m + 1e-12) * (r -l) + l


@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:], nb.int64[:])) (nb.int64[:], nb.float64[:,:]),
          
          ], cache = True, nogil = True)
def sum_by_group_multiple_values_return_num_appearance(groups, values):
    #values shape n_values * n_items
    order = np.argsort(groups)

    groups = groups[order]
    values = values[:, order]
    values = cumsum_2d(values)

    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    nodes = np.hstack((np.array([0]), np.where(index)[0] + 1))

    values = values[:, index]

    groups = groups[index]
    values[:, 1:] = values[:, 1:] - values[:, :-1]
    return groups, values, nodes[1:] - nodes[:-1]



@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:, :], nb.int64[:])) (nb.int64[:], nb.float64[:,:], nb.int64[:]),
          ], cache = True, nogil = True)
def sum_by_group_exclude_multiple_values_return_num_appearance(groups, values, exclude):
    #remove existed items
    inclusive = np.ones(len(groups), 'bool')
    for e in exclude:
      inclusive = inclusive & (groups != e)
    #values shape n_values * n_items
    return  sum_by_group_multiple_values_return_num_appearance(groups[inclusive], values[:, inclusive])

    

    
@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:,:])) (nb.int64[:], nb.float64[:,:,:]),
          ], cache = True, nogil = True)
def sum_by_group_multiple_values_2d(groups, values):
    order = np.argsort(groups)

    groups = groups[order]
    #first axis of values  is 3 types of actions , second axisis heterogenous, third axis is different items
    values = values[:, :, order]
    for i in prange(values.shape[0]):
      values[i] = cumsum_2d(values[i])

    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]

    values = values[:, :, index]

    groups = groups[index]
    values[:, :, 1:] = values[:, :,1:] - values[:, :,:-1]

    return groups, values

@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:,:])) (nb.int64[:], nb.float64[:,:,:]),
          ], cache = True, nogil = True)
def max_by_group_multiple_values_2d(groups, values):
    order = np.argsort(groups)

    groups = groups[order]
    #first axis of values  is 3 types of actions , second axisis heterogenous, third axis is different items
    values = values[:, :, order]
    

    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    groups = groups[index]
    ends = np.where(index)[0]

    k = 0
    for i in prange(1, values.shape[2]):
        if i != ends[k] + 1:
            values[:, :, i] = np.maximum(values[:, :, i], values[:, :, i - 1])
        else:
            k+=1
    
    values = values[:, :, index]

    return groups, values

@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.float64[:], nb.int64, nb.float64),
          nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.int64[:], nb.int64, nb.float64),
          # nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.float64[:], nb.float64[:], nb.int64, nb.float64),
          ], cache = True, nogil = True)
def find_topk_user(cousers, cofitness, min_topk_user, cofitness_thres):
  #sort cousers ascending by id
  order = np.argsort(cousers)
  cousers = cousers[order]
  cofitness = cofitness[order]

  #calculate groupby sum of cousers
  cumcofitness = np.cumsum(cofitness)

  #find unique
  index = np.ones(len(cousers), 'bool')

  #this is the last index of each group
  index[:-1] = cousers[1:] != cousers[:-1]
  cumcofitness = cumcofitness[index]
  

  #find start and end of groups
  nodes = np.hstack((np.array([0]), np.where(index)[0] + 1))

  starts = nodes[:-1]
  ends = nodes[1:]
  lens = ends - starts
  rank_nums = get_rank(lens)
    
  #caluculate groupby sum of cousers
  cumcofitness[1:] = cumcofitness[1:] - cumcofitness[:-1]

  rank_fit = get_rank(cumcofitness)
  
  order_fit = np.argsort(-(rank_nums + rank_fit))

  k = min_topk_user
  if cofitness_thres > 0:
    #get topk user
    #dynamic top user
    total_fitness = np.sum(cumcofitness)
    
    k = 0
    remain = 0
    thresh_hold = cofitness_thres * total_fitness
    while remain < thresh_hold:
      remain += cumcofitness[order_fit][k]
      k+=1 
    
    if k < min_topk_user:
      k = min_topk_user

    if k > 500:
      k = 500

  
  order_fit = order_fit[:k]

  starts = nodes[order_fit]
  ends = nodes[order_fit + 1]
  
  filter = np.zeros(len(cousers), 'bool')
  for i in prange(len(starts)):
    filter[starts[i] : ends[i]] = 1
    cofitness[starts[i] : ends[i]] = cofitness[starts[i] : ends[i]] * (ends[i] - starts[i])
  return cousers[filter], cofitness[filter], order, filter





@nb.njit(nb.types.Tuple((nb.int64[:], nb.float64[:],
                         nb.int64[:], nb.int64[:],
                         nb.int64[:], nb.int64[:],
                         nb.float64[:], nb.float64[:],
                         nb.float64[:]))(nb.int64[:, :], nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.int64) , cache = True, nogil = True)
def _get_test_items(edges, beta, test_click_weight, test_cart_weight, test_order_weight_l, test_order_weight_s, day):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder 6:susbsession, 7 fts
  
  time_decay = beta ** reverse_normalize(edges[:, 2])

  order_weight = np.where(edges[:, 5] > 1, test_order_weight_l, test_order_weight_s)

  weight = edges[:, 3] * test_click_weight + edges[:, 4] * test_cart_weight + order_weight 

  fitness =  time_decay * weight 

  order = np.argsort(-fitness)

  num_subsession = decode_2d(edges[ :, 6])
  return edges[order, 1], fitness[order], edges[order, 3] , edges[order, 4], edges[order, 5], \
          num_subsession[order], time_decay[order], normalize(edges[order,2]), normalize(edges[order, 7])

In [33]:
@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:, :]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.float64, nb.int64, nb.float64, nb.float64[:,:], nb.float64[:,:] , nb.float64[:,:], nb.int64), 
          nb.types.Tuple((nb.int64[:], nb.float64[:,:, :]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.int64,   nb.int64, nb.float64, nb.int64[:,:], nb.int64[:,:] , nb.int64[:,:], nb.int64), 
          ], cache = True, nogil = True)
def _get_item_neighbours(edges, lts, fts, prevsub, prev_click, prev_cart, prev_order, alpha, click_weight, cart_weight, order_weight, day):
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order, 7 -> subsession

  cosubsession = prevsub & edges[:, 7]
  cosubsession = decode_2d(cosubsession)

  reverse_user_variety = 1  / (1 + np.log(np.sum(decode_2d(edges[:, 7])) + 1))
  reverse_user_consistency = 1 / (1 + np.log((np.amax(edges[:,2]) - np.amin(edges[:, 3])) + 1))

  coclick = np.minimum(edges[:, 4], prev_click)
  cocart = np.minimum(edges[:, 5], prev_cart)
  coorder = np.minimum(edges[:, 6], prev_order)

  #weight 3 x n, line 0 find click, 1 find cart, 2 find order
  weight = coclick * click_weight + cocart * cart_weight  + coorder * order_weight 

  #check if union, if yes time diff = 0, else = min(fts1 - lts2, fts2 - lts1)
  is_end_later = edges[:, 2] > lts

  is_union = np.empty(edges.shape[0], dtype = np.bool_)
  is_union[is_end_later] =  edges[is_end_later, 3] < lts
  is_end_sooner = ~is_end_later
  is_union[is_end_sooner] = edges[is_end_sooner, 2] > fts

  
  subsession_coef = 1.8 ** np.clip(cosubsession, 0, 1)

  time_diff = np.empty(edges.shape[0],  dtype = np.float32) 
  
  time_diff[is_union] = 0

  not_union_and_later = ~is_union & is_end_later
  not_union_and_sooner = ~is_union & is_end_sooner


  time_diff[not_union_and_later] = np.abs(lts - edges[not_union_and_later, 3]) / day
  time_diff[not_union_and_sooner] = np.abs(fts - edges[not_union_and_sooner, 2]) / day
  time_decay = alpha ** time_diff


  fitness_subco = weight  * subsession_coef * reverse_user_variety * reverse_user_consistency 

  fitness_time_decay = weight * time_decay  * reverse_user_variety * reverse_user_consistency 

  #shape 3 x num_values x num_items
  values_to_return = np.stack((
      fitness_subco, fitness_time_decay, np.stack((cosubsession,cosubsession,cosubsession)),
      np.stack((coclick,coclick,coclick)), np.stack((cocart,cocart,cocart)), np.stack((coorder,coorder,coorder))
  ), axis = 1)
  
  return sum_by_group_multiple_values_2d(edges[:, 1], values_to_return)

# Step

In [34]:
def get_type_items(user, clicks, carts, orders, lens, rerank_tracking, train_data_clicks= None, train_data_carts= None, train_data_orders= None, num_cands = None,
                   test_train_data = None, test_num_cands = None, get_train = False):

  #get item in tests
  global ALPHA, BETA, MIN_TOPK_COUSER, COFITNESS_THRES, XGBOOST, GET_TEST_TRAIN, XGBOOST_TEST

  #get user features
  if XGBOOST or get_train or GET_TEST_TRAIN: 
    ori_user_features = get_user_features(user)
  
  #fitness sorted descending
  items, fitness, num_clicks, num_carts, num_orders, \
   num_subsession, time_decay, lts, fts = get_test_items(user, BETA)

  if GET_TEST_TRAIN:
    n = n = min(MAX_NUM_CANDS, len(items))
    if n > 20 or (not XGBOOST):
      
      user_features = []
      
      for f in ori_user_features:
        user_features.append(np.full(n, f))

      item_features = get_item_features(items[:n])

      test_train_data.append(np.transpose(np.stack((np.full(n, user),
                                          items[:n],
                                          fitness[:n],
                                          num_clicks[:n],
                                          num_carts[:n],
                                          num_orders[:n],
                                          num_subsession[:n],
                                          time_decay[:n],
                                          lts[:n],
                                          fts[:n],
                                          *user_features, 
                                          *item_features, 
                                          ))))
      test_num_cands.append(np.full(n, n))
  

  lens.append(len(items))
  #level 2 rerank
  need_rerank = False

  if len(items)  > 19:
    clicks.append(items[:20].tolist())
    carts.append(items[:20].tolist())
    orders.append(items[:20].tolist())

  else:
    num_more = 20 - len(items)
    
    
    
    #get couser
    coalls = [get_user_neighbours(items[i]) for i in range(len(items))]    
    cofitness = np.concatenate([np.full_like(all[0], fitness[i]) for i, all in enumerate(coalls)] )

    #find topk cousers
    cousers, cofitness, order, filter  = find_topk_user(np.concatenate([all[0] for all in coalls]), cofitness, MIN_TOPK_COUSER, COFITNESS_THRES)
    coltss = np.concatenate([all[1] for all in coalls])[order][filter]
    coftss = np.concatenate([all[2] for all in coalls])[order][filter]
    coclick = np.concatenate([all[3] for all in coalls])[order][filter]
    cocart = np.concatenate([all[4] for all in coalls])[order][filter]
    coorder = np.concatenate([all[5] for all in coalls])[order][filter]
    cosubsession = np.concatenate([all[6] for all in coalls])[order][filter]

    

    #get coitems
    coalls = [get_item_neighbours(cousers[i], coltss[i], coftss[i], cosubsession[i], coclick[i], cocart[i], coorder[i], ALPHA) for i in range(len(cousers))]

    #get each action type prediction
    for j, (a, train_data_a) in enumerate(zip([clicks, carts, orders], [train_data_clicks, train_data_carts, train_data_orders])):

      #multiply item by fitness of user which interacted with it
      
      this_action_cofitness = np.concatenate([all[1][j]* cofitness[i] for i, all in enumerate(coalls)], axis = -1) 
      coitems = np.concatenate([all[0] for all in coalls])
      #count covisit
      coitems, this_action_cofitness, num_appearance= sum_by_group_exclude_multiple_values_return_num_appearance(coitems, this_action_cofitness, exclude =  items)
      
      cofitness_cosub, cofitness_time_decay, \
      num_cosub,\
      coclick, cocart, coorder = (this_action_cofitness[i] for i in range(this_action_cofitness.shape[0]))
      #sort by count then to fitness

      rank_cosub = get_rank(cofitness_cosub)
      rank_time_decay = get_rank(cofitness_time_decay)
      rank_num_appearance = get_rank(num_appearance)


      
      order = np.argsort( -(rank_cosub  + rank_time_decay + rank_num_appearance * 0.2) )
      
      if get_train:
        
        if len(coitems):
          if j == 0:
            # n = min(30, len(coitems))
            n = min(MAX_NUM_CANDS, len(coitems))
            num_cands.append(np.full(n, n))
          else:
            n = min(MAX_NUM_CANDS, len(coitems))
            


          need_rerank = True
          

          aggs = ['mean', 'var']
          features = [cofitness_cosub[order[:n]],
                      cofitness_time_decay[order[:n]],
                      num_appearance[order[:n]],
                      num_cosub[order[:n]],
                      coclick[order[:n]],
                      cocart[order[:n]],
                      coorder[order[:n]]]
          
          
          #quotient features
          quo_features = []
          agg_features = []
          for f in features:
            for agg in aggs: 
              agg_val = getattr(f,agg)()
              agg_features.append(np.full(n, agg_val))
              quo_features.append(f / (agg_val + 1))
              
          
          
          user_features = []
          
          for f in ori_user_features:
            user_features.append(np.full(n, f))

          item_features = get_item_features(coitems[order[:n]])
          
          prod_features = []
          for i in range(len(features)):
            for i1 in range(i + 1, len(features)):
              prod_features.append(features[i] * features[i1])
          
          #rank features 
          rank_features = [
            get_rank(-f) for f in [*features, *item_features, *quo_features, *prod_features]
          ]

                
          train_data_a.append(np.transpose(np.stack((np.full(n, user),
                                                  coitems[order[:n]],
                                                  np.full(n, j), #action
                                                  *features,
                                                  *item_features, 
                                                  *quo_features,
                                                  *prod_features,
                                                  *rank_features,
                                                  *agg_features,
                                                  *user_features,
                                                  ))))
          

      if XGBOOST: 
        # news = items.tolist() +  coitems[order[:num_more]].tolist() 
        # a.append(news)
        a.append(items.tolist())
      else:
        news = items.tolist() +  coitems[order[:num_more]].tolist() 
        a.append(news)
  rerank_tracking.append(need_rerank)
    
    

    


# XGBoost Rerank

In [35]:
from xgb_utils import create_data, create_test_data

In [36]:
def xgboost_infer(infer_data, num_cands, perm, models):
  df_infer_data, np_type = create_data(infer_data)
  preds = np.zeros(df_infer_data.shape[0])
  dtest = xgb.DMatrix(data=df_infer_data.iloc[:, 2:])

  for model in models:

      preds += model.predict(dtest) / len(models)
      
  predictions = df_infer_data[['user','item']].copy()
  predictions['type'] = np_type
  predictions['pred'] = preds

  predictions = predictions.sort_values(['user', 'pred'], ascending=[True, False]).reset_index(drop=True)
  predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
  predictions = predictions.loc[predictions.n<20]
  sub = predictions.groupby('user').item.apply(list)
  sub = sub.to_frame().reset_index()
  items = sub.item.values.tolist()
  users = sub.user.values
  unshuffle([items, users], perm)
  return items, users
    

In [37]:
def xgboost_test_infer(infer_data, num_cands, perm, models):
  # global df_infer_data
  df_infer_data = create_test_data(infer_data)
  preds = np.zeros(df_infer_data.shape[0])
  dtest = xgb.DMatrix(data=df_infer_data.iloc[:, 2:])

  for model in models:

      preds += model.predict(dtest) / len(models)
      
  predictions = df_infer_data[['user','item']].copy()
  predictions['pred'] = preds

  predictions = predictions.sort_values(['user', 'pred'], ascending=[True, False]).reset_index(drop=True)
  predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
  predictions = predictions.loc[predictions.n<20]
  sub = predictions.groupby('user').item.apply(list)
  sub = sub.to_frame().reset_index()
  items = sub.item.values.tolist()
  users = sub.user.values
  unshuffle([items, users], perm)
  return items, users
    

# Batchproc and XGBoost postprocess


In [38]:
import multiprocessing
import xgboost as xgb

def batch_proc(*args):
  global GET_TRAIN, XGBOOST, GET_RESULT
  batch, batch_name = args[0]
  #create a random 0 to prevent numpy cast clicks into 2d array instead of 1d array of list 
  clicks = [0]
  carts = [0]
  orders = [0]
  lens = []
  num_cands = []
  rerank_tracking = []
  train_data_clicks = []
  train_data_carts = []
  train_data_orders = []
  test_num_cands = []
  test_train_data = []
  for user in batch:
    get_type_items(user, clicks, carts, orders, lens, rerank_tracking, train_data_clicks, train_data_carts, train_data_orders, num_cands, test_train_data, test_num_cands, GET_TRAIN)

  if XGBOOST:
    clicks = np.array(clicks, dtype='object')
    carts = np.array(carts, dtype='object')
    orders = np.array(orders, dtype='object')
  
  #remove the initial blank list
  clicks = clicks [1:]
  carts = carts[1:]
  orders = orders[1:]

  result = []
  if GET_RESULT:
    result.append( [[clicks, carts, orders, lens], CACHE + f'/result_batch_{batch_name}'])
    
  if GET_TRAIN:
    train_data = []
    for data in [train_data_clicks, train_data_carts, train_data_orders]:
      train_data.extend(data)
      
    result.append([ [ train_data, num_cands], CACHE + f'/train_data_{batch_name}'])
  
  if GET_TEST_TRAIN:
    result.append( [ [test_train_data, test_num_cands], CACHE + f'/test_train_data_{batch_name}'])
  
  if XGBOOST:
    result.append(np.array(rerank_tracking))
  return result, batch
    

In [39]:
import traceback
def postprocess(rs, batch, test = False):
  if XGBOOST: 
    result_file = rs[0][1]
    clicks, carts, orders, lens = rs[0][0]
    try:
      infer_data, num_cands = rs[1][0]
      rerank_tracking = rs[3]
      assert len(infer_data) / 3 == rerank_tracking.sum(), (len(infer_data), len(rerank_tracking))
      num_cands = np.concatenate(num_cands)
      num_features = infer_data[0].shape[-1]

      infer_data = np.concatenate(infer_data).reshape(3, -1, num_features)
      infer_data = [infer_data[i] for i in range(3)]
    except:
      pass
      print(traceback.format_exc())

    over_lens = np.where(np.array(lens) > 20)[0]
    infer_test_data, test_num_cands = rs[2][0]
    assert len(infer_test_data) == len(over_lens), (len(infer_test_data), len(over_lens))
    num_features = infer_test_data[0].shape[-1]
    infer_test_data = np.concatenate(infer_test_data).reshape(-1, num_features)
    #rerank test items
    
    if len(over_lens): 
      user = batch [over_lens] 
      p = np.argsort(user)
      suggest = [clicks[over_lens], carts[over_lens], orders[over_lens]]

      for i, models in enumerate([test_click_models, test_cart_models, test_order_models]):
        rerank, u = xgboost_test_infer(infer_test_data, num_cands, p, models)

        assert np.array_equal(u, user), (u[:10], user[:10])
        for j in range(len(suggest[i])):
          suggest[i][j] = rerank[j][:20]

      clicks[over_lens] = suggest[0]
      carts[over_lens] = suggest[1]
      orders[over_lens] = suggest[2]
      

    #rerank level 2 items
    # if False:
    if len(rerank_tracking):
      assert len(clicks.shape) == 1
      assert len(rerank_tracking.shape) == 1, len(rerank_tracking.shape)

      user = batch[rerank_tracking]
      p = np.argsort(user)
      
      suggest = [clicks[rerank_tracking], carts[rerank_tracking], orders[rerank_tracking]]

      for i, (data, models) in enumerate(zip(infer_data, [click_models, cart_models, order_models])):
        rerank, u = xgboost_infer(data, num_cands, p, models)

        assert np.array_equal(u, user), (u[:10], user[:10])
        for j in range(len(suggest[i])):
          suggest[i][j] = (suggest[i][j] +  rerank[j])[:20]

      rerank_tracking = np.where(rerank_tracking)[0]
      clicks[rerank_tracking] = suggest[0]
      carts[rerank_tracking] = suggest[1]
      orders[rerank_tracking] = suggest[2]

    if not test:
      with open(result_file, 'wb') as f: 
        pickle.dump([clicks.tolist(), carts.tolist(), orders.tolist(), lens], f)

  if WRITE_FILE and (not test):
    
    for data, fn in rs: 
        with open(fn, 'wb') as f: 
          pickle.dump(data, f)

# Infer Params

In [40]:
import multiprocessing
CACHE = '/home/anhphantq/vdb/cache_lb'

MODEL_PATH = '/home/anhphantq/xgboost'
MODEL_VERSION = '4'
TEST_MODEL_PATH = '/home/anhphantq/xgboost_test'
TEST_MODEL_VERSION = '2'

#TO use xgboost enable get_train, get_result, get_test_train and disable writefile
GET_TRAIN = True
GET_TEST_TRAIN = True
GET_RESULT = True
WRITE_FILE = False
XGBOOST = True



NUM_CPU = multiprocessing.cpu_count()
BATCH_SIZE = 1000

In [41]:
!rm -rf {CACHE}
!mkdir {CACHE}

# Create data batches

In [43]:
def shuffle(l):  
    perm = np.random.permutation(len(l))  
    l[:] = [l[j] for j in perm]  
    return perm

def unshuffle(L, perm):  
    assert len(L[0]) == len(perm), (len(L[0]), len(perm))
    res = [[None] * len(l) for l in L]
    for i, j in enumerate(perm):
        for k, l in enumerate(L):
          res[k][j] = l[i]
          
    for i, l in enumerate(L):
      l[:] = res[i]

In [44]:
import _pickle as pickle
perm = shuffle(session_list)
with open(CACHE + '/perm', 'wb') as f:
  pickle.dump(perm, f)

In [45]:
import _pickle as pickle

#divide into batches

num_batches = len(session_list) // BATCH_SIZE

start = np.arange(num_batches) * BATCH_SIZE
end = start + BATCH_SIZE
end[-1] = len(session_list)
batches = [session_list[start[i] : end[i]] for i in range(len(start))]
batch_names = [i for i in range(len(batches))]

with open(CACHE + '/metadata', 'wb') as f:
  pickle.dump({
      'num_frags': len(batches)
  }, f)

# Load XGB models

In [46]:
import xgboost as xgb
click_models = []
for fold in range(3):
  click_model = xgb.Booster()
  click_model.load_model(MODEL_PATH + '/clicks_only_' + MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  click_models.append(click_model)

cart_models = []
for fold in range(3):
  cart_model = xgb.Booster()
  cart_model.load_model(MODEL_PATH + '/carts_only_'+ MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  cart_models.append(cart_model)

order_models = []
for fold in range(3):
  order_model = xgb.Booster()
  order_model.load_model(MODEL_PATH + '/orders_only_'+ MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  order_models.append(order_model)

In [47]:
import xgboost as xgb
test_click_models = []
for fold in range(3):
  test_click_model = xgb.Booster()
  test_click_model.load_model(TEST_MODEL_PATH + '/clicks_only_' + TEST_MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  test_click_models.append(test_click_model)

test_cart_models = []
for fold in range(3):
  test_cart_model = xgb.Booster()
  test_cart_model.load_model(TEST_MODEL_PATH + '/carts_only_'+ TEST_MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  test_cart_models.append(test_cart_model)

test_order_models = []
for fold in range(3):
  test_order_model = xgb.Booster()
  test_order_model.load_model(TEST_MODEL_PATH + '/orders_only_'+ TEST_MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  test_order_models.append(test_order_model)

In [52]:
# xgb.plot_importance(test_click_model)
# xgb.plot_importance(test_cart_model)
# xgb.plot_importance(order_models[-1])

# Test pipeline

In [50]:
from tqdm import tqdm
clicks = []
carts = []
orders = []
lens = []
train_data =[]
num_cands = []
# for user in tqdm(np.random.choice(session_list, size = 1000)):
rs, batch = batch_proc((batches[0], [0]))
postprocess(rs, batch, test = True) 


# Execute Multiprocessing

In [48]:
%%time
from tqdm import tqdm
import _pickle as pickle

pool = multiprocessing.Pool(NUM_CPU)

for rs, batch in tqdm(pool.imap_unordered(batch_proc, tuple(zip(batches, batch_names)), chunksize = 1), total = num_batches):
  postprocess(rs, batch)



 41%|████      | 677/1671 [54:23<1:04:47,  3.91s/it]

: 

: 

In [ ]:
pool.close()
pool.join()


# Load result from pickle

In [ ]:
CACHE = '/home/anhphantq/vdb/cache_lb'


In [ ]:


def unshuffle(L, perm):  
    
    res = [[None] * len(l) for l in L]
    for i, j in enumerate(perm):
        for k, l in enumerate(L):
          res[k][j] = l[i]
          
    for i, l in enumerate(L):
      l[:] = res[i]

In [ ]:
import _pickle as pickle
with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = metadata['num_frags']

In [ ]:
import multiprocessing
from tqdm import tqdm

clicks = []
carts = []
orders = []
lens = []
for i in tqdm(range(num_frags)):
    with open(CACHE + f'/result_batch_{i}', 'rb') as f:
      result = pickle.load(f)
    
    clicks.extend(result[0])
    carts.extend(result[1])
    orders.extend(result[2])
    lens.extend(result[3])
    


100%|██████████| 90/90 [00:00<00:00, 121.99it/s]


In [ ]:
del result

In [ ]:
import _pickle as pickle
with open(CACHE + '/perm', 'rb') as f:
  perm = pickle.load(f)
# unshuffle([clicks,carts,orders,lens, session_list] + [train_data] if GET_TRAIN else [], perm)
unshuffle([clicks,carts,orders,lens, session_list], perm)
# unshuffle([clicks,carts,orders], perm)

# Create submission

In [ ]:
import pandas as pd
count = 0
num_all = 0
missing = []
for preds in [clicks, carts, orders]:
  for i, p in enumerate(preds):
    if len(p) != 20:
      missing.append(i)
      count+= 20 - len(p)
    num_all +=1

len(missing), count, num_all

(813, 9261, 270186)

In [ ]:
click_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in clicks] if JOIN else [[int(j) - 100000000 for j in i] for i in clicks]
cart_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in carts] if JOIN else [[int(j) - 100000000 for j in i] for i in carts]
order_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in orders] if JOIN else [[int(j) - 100000000 for j in i] for i in orders]

In [ ]:
submission1 = pd.DataFrame({'session_type': session_list, 'labels': click_new_append})
submission2 = pd.DataFrame({'session_type': session_list, 'labels': cart_new_append})
submission3 = pd.DataFrame({'session_type': session_list, 'labels': order_new_append})

In [ ]:
submission1['session_type'] = submission1['session_type'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission2['session_type'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission3['session_type'].apply(lambda x: str(x) + '_orders')

In [ ]:
final = pd.concat([submission1, submission2, submission3])

In [ ]:
del submission1, submission2, submission3

In [ ]:
final

,session_type,labels
0,11098528_clicks,"[11830, 588923, 1157882, 307904, 571762, 20535..."
1,11098529_clicks,"[1105029, 612829, 1049489, 217742, 459126, 133..."
2,11098530_clicks,"[409236, 264500, 583026, 1603001, 1058185, 254..."
3,11098531_clicks,"[1271998, 1728212, 1365569, 396199, 452188, 15..."
4,11098532_clicks,"[876469, 7651, 108125, 1202618, 1159379, 14025..."
...,...,...
90057,11188585_orders,"[1653393, 115404, 169239, 1176975, 823801, 157..."
90058,11188586_orders,"[871551, 1531151, 489812, 864925, 187937, 2149..."
90059,11188587_orders,"[416407, 409907, 94130, 1083413, 974644, 17390..."
90060,11188588_orders,"[1663535, 1347624, 982423, 1582485, 576116, 78..."


# Eval valid

In [98]:
import pandas as pd
def get_score(df, split = False):
    submission = df.copy()
    test_labels = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
    test_labels = test_labels[(test_labels['session'] <= np.amax(session_list)) & (test_labels['session'] >= np.amin(session_list))]
    
    submission['session'] = submission.session_type.apply(lambda x: int(x.split('_')[0]))
    
    session = set(test_labels['session'].unique().tolist())
    for j in session_list:
      assert j in session


    submission['type'] = submission.session_type.apply(lambda x: x.split('_')[1])
    
    if split:
      submission.labels = submission.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

    test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)

    del submission

    recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 

    print (f"Score : {(recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()}")
    return recall_per_type

In [99]:
get_score(final, JOIN)

Score : 0.5496198932449281


type
carts     0.391780
clicks    0.507641
orders    0.635536
dtype: float64

# Save sumission

In [ ]:
final.to_csv('submission.csv', index = False)

#Submit


In [ ]:
!kaggle competitions submit -c otto-recommender-system -f submission.csv -m "hehe"

100%|████████████████████████████████████████| 783M/783M [00:42<00:00, 19.2MB/s]
Successfully submitted to OTTO – Multi-Objective Recommender System

#Submit 1

In [ ]:
!pip install kaggle

In [ ]:
%%bash
rm ~/.kaggle/kaggle.json
mkdir ~/.kaggle
cp /content/drive/MyDrive/ColabNotebooks/kaggle1.json ~/.kaggle
mv ~/.kaggle/kaggle1.json ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c otto-recommender-system -f /content/drive/MyDrive/ColabNotebooks/12_9_2_submission.csv -m "validate mega graph tool test sampling"

In [ ]:
preds[0]

In [ ]:
count = 0
missing = []
for i, p in enumerate(preds):
  if len(p) != 20:
    missing.append(i)

count

In [ ]:
new_append = [' '.join([str(int(j) - 100000000) for j in i) for m, i in enumerate(preds)] if JOIN else [[int(j) - 100000000 for j in i[ : lens[m]]] for m, i in enumerate(preds)]

In [ ]:
submission1 = test_df.copy()
submission1['labels'] = new_append
submission2 = submission1.copy()
submission3 = submission1.copy()

In [ ]:
submission1

In [ ]:
submission1['session_type'] = submission1['session'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission1['session'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission1['session'].apply(lambda x: str(x) + '_orders')

In [ ]:
final = pd.concat([submission1, submission2, submission3])

In [ ]:
del submission1, submission2, submission3

In [ ]:
del final['session'], final['ts']


In [ ]:
final = final.droplevel(1, axis = 1)

In [ ]:
final

In [ ]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_9_3_submission.csv', index = False)

# Create submission test only 

In [ ]:
count = 0
num_all = 0
missing = []
for preds in [clicks, carts, orders]:
  for i, p in enumerate(preds):
    if len(p) != 20:
      missing.append(i)
      count+= 20 - len(p)
    num_all +=1

len(missing), count, num_all

(810, 9240, 270186)

In [ ]:
click_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : lens[m]]]) for m, a in enumerate(clicks)] if JOIN else [[int(j) - 100000000 for j in i[ : lens[m]]] for m, i in enumerate(clicks)]
cart_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : lens[m]]]) for m, a in enumerate(carts)] if JOIN else [[int(j) - 100000000 for j in i[ : lens[m]]] for m, i in enumerate(carts)]
order_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : lens[m]]]) for m, a in enumreate (orders)] if JOIN else [[int(j) - 100000000 for j in i[ : lens[m]]] for m, i in enumerate(orders)]

In [ ]:
submission1 = pd.DataFrame({'session_type': session_list, 'labels': click_new_append})
submission2 = pd.DataFrame({'session_type': session_list, 'labels': cart_new_append})
submission3 = pd.DataFrame({'session_type': session_list, 'labels': order_new_append})

In [ ]:
submission1['session_type'] = submission1['session_type'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission2['session_type'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission3['session_type'].apply(lambda x: str(x) + '_orders')

In [ ]:
submission1

,session_type,labels
0,11110782_clicks,[1006198]
1,11141605_clicks,[390163]
2,11155083_clicks,[691379]
3,11169363_clicks,"[1210932, 740069, 177098]"
4,11123184_clicks,[1505419]
...,...,...
90057,11125940_clicks,"[131413, 686372]"
90058,11161626_clicks,[335442]
90059,11129829_clicks,"[725261, 403756, 1188457, 1381296, 456361, 133..."
90060,11184001_clicks,[38052]


In [ ]:
final = pd.concat([submission1, submission2, submission3])

In [ ]:
del submission1, submission2, submission3

In [ ]:
final

,session_type,labels
0,11110782_clicks,[1006198]
1,11141605_clicks,[390163]
2,11155083_clicks,[691379]
3,11169363_clicks,"[1210932, 740069, 177098]"
4,11123184_clicks,[1505419]
...,...,...
90057,11125940_orders,"[131413, 686372]"
90058,11161626_orders,[335442]
90059,11129829_orders,"[725261, 403756, 1188457, 1381296, 456361, 133..."
90060,11184001_orders,[38052]


# Merge rerank

In [ ]:
rerank = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/test_cart_rerank.csv')

In [ ]:
rerank['labels'] = rerank['labels'].apply(lambda x: [int(i) for i in x.split(' ')])

In [ ]:
rerank

,session_type,labels
0,11098528_carts,"[588923, 571762, 1157882, 735729, 884502, 2053..."
1,11098529_carts,"[1339838, 1049489, 459126, 1544564, 612829, 53..."
2,11098530_carts,"[1603001, 583026, 254154, 877496, 963957, 1058..."
3,11098531_carts,"[1758580, 1206090, 71884, 1630025, 653835, 605..."
4,11098532_carts,"[108125, 1202618, 1557927, 1402537, 1159379, 7..."
...,...,...
88007,11188585_carts,"[169239, 1176975, 1734926, 1574899, 260496, 17..."
88008,11188586_carts,"[864925, 759258, 214934, 1072157, 1144361, 200..."
88009,11188587_carts,"[316498, 1192114, 197808, 243636, 672432, 7929..."
88010,11188588_carts,"[493104, 479834, 1582485, 1682122, 29735, 6965..."


In [ ]:
final

,session_type,labels
0,11110782_clicks,[1006198]
1,11141605_clicks,[390163]
2,11155083_clicks,[691379]
3,11169363_clicks,"[1210932, 740069, 177098]"
4,11123184_clicks,[1505419]
...,...,...
90057,11125940_orders,"[131413, 686372]"
90058,11161626_orders,[335442]
90059,11129829_orders,"[725261, 403756, 1188457, 1381296, 456361, 133..."
90060,11184001_orders,[38052]


In [ ]:
final = final.merge(rerank, on = 'session_type', how = 'left')

In [ ]:
final

,session_type,labels_x,labels_y
0,11110782_clicks,[1006198],NaN
1,11141605_clicks,[390163],NaN
2,11155083_clicks,[691379],NaN
3,11169363_clicks,"[1210932, 740069, 177098]",NaN
4,11123184_clicks,[1505419],NaN
...,...,...,...
270181,11125940_orders,"[131413, 686372]",NaN
270182,11161626_orders,[335442],NaN
270183,11129829_orders,"[725261, 403756, 1188457, 1381296, 456361, 133...",NaN
270184,11184001_orders,[38052],NaN


In [ ]:
labels = []
for row in final.itertuples():
  labels.append(row.labels_x + row.labels_y if not type(row.labels_y) == float else row.labels_x)

In [ ]:
final['labels'] = labels
del final['labels_x'], final['labels_y']

In [ ]:
final.loc[100000]

session_type                                       11116226_carts
labels_x                                          [265992, 60347]
labels_y        [166037, 1022566, 544144, 343267, 201568, 4485...
labels          [265992, 60347, 166037, 1022566, 544144, 34326...
Name: 100000, dtype: object

# Save sumission

In [ ]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_9_5_submission.csv', index = False)